# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>1º Fase</font>

### <font color='blue'>Predição de pregões futuros de ativos que compõem o índice SP500.</font>

![title](data\image\logo.jpeg)

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

In [ ]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Bibliotecas e Frameworks

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import *
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.losses import MeanSquaredError
from tensorboard import notebook
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time
from datetime import datetime
import os

In [ ]:
# Versões dos pacotes usados neste jupyter notebook

%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

In [ ]:
sns.set_style('whitegrid')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
#Confirma se o TensorFlow pode acessar a GPU

device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
    
print('Found GPU at: {}'.format(device_name))

In [ ]:
# Estado da GPU

!nvidia-smi

In [ ]:
# Parametros fixos de treinamento

verbose = 2
seed = 25
steps = 30
epochs = 2000
batch_size = 32
logRetPeriod = 20
graphic = True
downloadData = False
trainModel = True

np.random.seed(seed)

In [ ]:
forecast = ['2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', 
            '2022-10-31', '2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04', 
            '2022-11-07', '2022-11-08', '2022-11-09', '2022-11-10', '2022-11-11',
            '2022-11-14', '2022-11-15', '2022-11-16', '2022-11-17', '2022-11-18']

In [ ]:

ativos = ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT',
          'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES',
          'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL',
          'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN',
          'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE',
          'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA',
          'BAC', 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB',
          'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRK.B', 'BRO',
          'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB',
          'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CEG',
          'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX',
          'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF',
          'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO',
          'CSGP', 'CSX', 'CTAS', 'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CVS',
          'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI',
          'DHR', 'DIS', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DPZ', 'DRI',
          'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'DXCM', 'EA', 'EBAY', 'ECL',
          'ED', 'EFX', 'EIX', 'EL', 'ELV', 'EMN', 'EMR', 'ENPH', 'EOG',
          'EPAM', 'EQIX', 'EQR', 'EQT', 'ES', 'ESS', 'ETN', 'ETR', 'ETSY',
          'EVRG', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG', 'FAST',
          'FBHS', 'FCX', 'FDS', 'FDX', 'FE', 'FFIV', 'FIS', 'FISV', 'FITB',
          'FLT', 'FMC', 'FOX', 'FOXA', 'FRC', 'FRT', 'FTNT', 'FTV', 'GD',
          'GE', 'GILD', 'GIS', 'GL', 'GLW', 'GM', 'GNRC', 'GOOG', 'GOOGL',
          'GPC', 'GPN', 'GRMN', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HCA',
          'HD', 'HES', 'HIG', 'HII', 'HLT', 'HOLX', 'HON', 'HPE', 'HPQ',
          'HRL', 'HSIC', 'HST', 'HSY', 'HUM', 'HWM', 'IBM', 'ICE', 'IDXX',
          'IEX', 'IFF', 'ILMN', 'INCY', 'INTC', 'INTU', 'INVH', 'IP', 'IPG',
          'IQV', 'IR', 'IRM', 'ISRG', 'IT', 'ITW', 'IVZ', 'J', 'JBHT', 'JCI',
          'JKHY', 'JNJ', 'JNPR', 'JPM', 'K', 'KDP', 'KEY', 'KEYS', 'KHC',
          'KIM', 'KLAC', 'KMB', 'KMI', 'KMX', 'KO', 'KR', 'L', 'LDOS', 'LEN',
          'LH', 'LHX', 'LIN', 'LKQ', 'LLY', 'LMT', 'LNC', 'LNT', 'LOW',
          'LRCX', 'LUMN', 'LUV', 'LVS', 'LW', 'LYB', 'LYV', 'MA', 'MAA',
          'MAR', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO', 'MDLZ', 'MDT', 'MET',
          'META', 'MGM', 'MHK', 'MKC', 'MKTX', 'MLM', 'MMC', 'MMM', 'MNST',
          'MO', 'MOH', 'MOS', 'MPC', 'MPWR', 'MRK', 'MRNA', 'MRO', 'MS',
          'MSCI', 'MSFT', 'MSI', 'MTB', 'MTCH', 'MTD', 'MU', 'NCLH', 'NDAQ',
          'NDSN', 'NEE', 'NEM', 'NFLX', 'NI', 'NKE', 'NLOK', 'NLSN', 'NOC',
          'NOW', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVDA', 'NVR', 'NWL',
          'NWS', 'NWSA', 'NXPI', 'O', 'ODFL', 'OGN', 'OKE', 'OMC', 'ON',
          'ORCL', 'ORLY', 'OTIS', 'OXY', 'PARA', 'PAYC', 'PAYX', 'PCAR',
          'PCG', 'PEAK', 'PEG', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH',
          'PHM', 'PKG', 'PKI', 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'POOL',
          'PPG', 'PPL', 'PRU', 'PSA', 'PSX', 'PTC', 'PWR', 'PXD', 'PYPL',
          'QCOM', 'QRVO', 'RCL', 'RE', 'REG', 'REGN', 'RF', 'RHI', 'RJF',
          'RL', 'RMD', 'ROK', 'ROL', 'ROP', 'ROST', 'RSG', 'RTX', 'SBAC',
          'SBNY', 'SBUX', 'SCHW', 'SEDG', 'SEE', 'SHW', 'SIVB', 'SJM', 'SLB',
          'SNA', 'SNPS', 'SO', 'SPG', 'SPGI', 'SRE', 'STE', 'STT', 'STX',
          'STZ', 'SWK', 'SWKS', 'SYF', 'SYK', 'SYY', 'T', 'TAP', 'TDG',
          'TDY', 'TECH', 'TEL', 'TER', 'TFC', 'TFX', 'TGT', 'TJX', 'TMO',
          'TMUS', 'TPR', 'TRMB', 'TROW', 'TRV', 'TSCO', 'TSLA', 'TSN', 'TT',
          'TTWO', 'TWTR', 'TXN', 'TXT', 'TYL', 'UAL', 'UDR', 'UHS', 'ULTA',
          'UNH', 'UNP', 'UPS', 'URI', 'USB', 'V', 'VFC', 'VICI', 'VLO',
          'VMC', 'VNO', 'VRSK', 'VRSN', 'VRTX', 'VTR', 'VTRS', 'VZ', 'WAB',
          'WAT', 'WBA', 'WBD', 'WDC', 'WEC', 'WELL', 'WFC', 'WHR', 'WM',
          'WMB', 'WMT', 'WRB', 'WRK', 'WST', 'WTW', 'WY', 'WYNN', 'XEL',
          'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS']

## Download dos Dados

In [ ]:
if downloadData:

    start_date = "2017-10-21"
    end_date = "2022-10-21"

    data = web.DataReader(name = '^GSPC', data_source = 'yahoo', start = start_date, end = end_date)
    SP500_index = pd.DataFrame(data['Close']).reset_index().rename(columns={'Close': 'SP500', 'Date': 'Dia'})

    SP500_close = pd.DataFrame()

    for ativo in ativos:
  
        if ativo == 'BF.B':
            ativo = 'BF-B'

        if ativo == 'BRK.B':
            ativo = 'BRK-B'

        data = web.DataReader(name = ativo, data_source = 'yahoo', start = start_date, end = end_date)
        temp_close = pd.DataFrame(data['Close'])
        SP500_close = pd.concat([SP500_close, temp_close], axis = 1)

        
    SP500_close.columns = ativos
    SP500_close.reset_index(inplace = True)
    SP500_close.rename(columns={'Date': 'Dia'}, inplace = True)

    assert SP500_close.isna().sum().mean() == 0,  "Valores Faltantes"
    assert SP500_index.isna().sum().mean() == 0,  "Valores Faltantes"

    SP500_close.to_csv(path_or_buf = 'data/SP500_close', index = False)
    SP500_index.to_csv(path_or_buf = 'data/SP500_index', index = False)
    
else:
    
    SP500_close = pd.read_csv('data/SP500_close')
    SP500_index = pd.read_csv('data/SP500_index')

In [ ]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

setores = {'Industrials': df.loc[df['GICS Sector'] == 'Industrials']['Symbol'].tolist(),
           'HealthCare': df.loc[df['GICS Sector'] == 'HealthCare']['Symbol'].tolist(),
           'InformationTechnology': df.loc[df['GICS Sector'] == 'InformationTechnology']['Symbol'].tolist(),
           'CommunicationServices': df.loc[df['GICS Sector'] == 'CommunicationServices']['Symbol'].tolist(),
           'ConsumerStaples': df.loc[df['GICS Sector'] == 'ConsumerStaples']['Symbol'].tolist(),
           'ConsumerDiscretionary': df.loc[df['GICS Sector'] == 'ConsumerDiscretionary']['Symbol'].tolist(),
           'Utilities': df.loc[df['GICS Sector'] == 'Utilities']['Symbol'].tolist(),
           'Financials': df.loc[df['GICS Sector'] == 'Financials']['Symbol'].tolist(),
           'Materials': df.loc[df['GICS Sector'] == 'Materials']['Symbol'].tolist(),
           'RealEstate': df.loc[df['GICS Sector'] == 'RealEstate']['Symbol'].tolist()}

## Pré-Processamento dos Dados

In [ ]:
def generatorTimeframeTable(table, ativo):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    
    TimeframeTable = pd.DataFrame(np.zeros((len(table[ativo])-steps, steps+1), dtype='float64'), columns = nameColumns)

    for index, close in enumerate(table[ativo]):
        tempA = index
        tempB = 0
        for i in range(steps+1):
            if tempA < len(table[ativo])-steps and tempA >=0:
                TimeframeTable.iloc[tempA, tempB] = close

            tempA -= 1
            tempB += 1

    timeIndex = table.iloc[steps:,0]
    TimeframeTable["Dia"] = timeIndex.to_numpy()
    TimeframeTable.set_index("Dia", inplace = True)
    
    return TimeframeTable

In [ ]:
def createTrainScaler(df):
    
    trainScaler = pd.DataFrame()
 
    for _ in range(steps+1):
        temp_close = pd.DataFrame(df.iloc[:,-1])
        trainScaler = pd.concat([trainScaler, temp_close], axis = 1)

    scaler = StandardScaler()

    scaler.fit(trainScaler)

    return scaler

In [ ]:
def preprocessingdata(steps, df, ativos):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    

    aux = []
    
    for ativo in ativos:
        trainDataAtivo = generatorTimeframeTable(df, ativo)
        trainDataAtivo.dropna(axis = 0, inplace = True)
        
        #----Score-Z--------------------------------------
        scaler = createTrainScaler(trainDataAtivo)
        trainDataAtivo = scaler.transform(trainDataAtivo)
        #-------------------------------------------------
        aux.append(trainDataAtivo)
    
    trainData = np.concatenate(tuple(aux), axis=0)
    
    X = trainData[:, :-1]
    y = trainData[:, -1]
    

    #------Divisão de dados entre Treino e Validação------------------------------------------------
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, shuffle = False)

    X_treino = X_treino.reshape((-1, steps, 1))
    X_teste = X_teste.reshape((-1, steps, 1))
    #-----------------------------------------------------------------------------------------------
    
    return [X_treino, X_teste, y_treino, y_teste]

## Construção, Treinamento e Avaliação do Modelo

In [ ]:
# Callbacks

tensorboard = TensorBoard(log_dir="logs/{}".format(datetime.now().strftime('%d-%B-%Ih%Mmin')))

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose = verbose,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='loss',
                              factor=0.2,
                              patience=3,
                              mode="min",
                              verbose = verbose,
                              min_delta=0.0001,
                              min_lr=0)

callbacks = [tensorboard, earlystop, reduce_lr, TerminateOnNaN()]

In [ ]:
def create_model():
     
    model = Sequential()
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True,
                   input_shape = (steps, 1)))  

    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True))  
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = False)) 
    
    model.add(Dense(1, activation = 'linear'))
    
    Lmse = MeanSquaredError()
    
    opt_Adadelta = Adadelta(learning_rate = 0.01, rho = 0.95, epsilon = 1e-07)

    model.compile(loss= Lmse, optimizer = opt_Adadelta)

    return model

In [ ]:
def fillTableFrame(ativo, tablePrevision, model, table = SP500_close):

    TimeframeTable = generatorTimeframeTable(table, ativo)
    
    index_data = TimeframeTable.index
    
    scaler = createTrainScaler(TimeframeTable)

    TimeframeTable = scaler.transform(TimeframeTable)
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))

    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    
    for day in forecast:
        
        current_info = TimeframeTable.iloc[-1, 1:].to_numpy()
        
        standardCurrentInfo = current_info.reshape(1, steps, 1).astype('float32')
        
        current_forecast = model.predict(standardCurrentInfo, verbose=False).reshape(1,)
        
        new_line = np.concatenate((current_info, current_forecast), axis = 0)
        
        TimeframeTable = pd.concat([TimeframeTable,
                                    pd.DataFrame(new_line.reshape(1, -1),
                                                 columns = nameColumns,
                                                 index = [day])], axis = 0)
        
        
    index_data = TimeframeTable.index  
    
    TimeframeTable = scaler.inverse_transform(TimeframeTable)
    
    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    TimeframeTable.index = pd.to_datetime(TimeframeTable.index)
    
    
    #--------Popula tabela de previsão---------------------------------------------------
    if ativo in ativos:
            for day in forecast:
                tablePrevision.loc[day, ativo] = TimeframeTable.loc[day, 'Close-0']
    #------------------------------------------------------------------------------------
   
    return TimeframeTable

In [ ]:
# Popula tabela de log-Retorno e gera gráficos

def fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model):
    
    lengthTable = len(tableLogRet)

    for ativo in setor:

        TimeframeSPAux = fillTableFrame(ativo, tablePrevision, model)

        #-----------Graphic------------------------------------------------------------------------------------------
        if graphic:
            
            outdir = './graphics/{}-{}'.format(nameSetor, datetime.now().strftime('%d-%B-%Ih%Mmin'))

            if not os.path.exists(outdir):
                os.mkdir(outdir)
            
            fig, ax = plt.subplots()
            ax.plot(TimeframeSPAux.index[:-len(forecast)], TimeframeSPAux.iloc[:-len(forecast), -1], linewidth=2.0, c = 'b')
            ax.plot(TimeframeSPAux.index[-len(forecast):], TimeframeSPAux.iloc[-len(forecast):, -1], linewidth=2.0, c = 'r', ls = '-')
            ax.legend(['Atual', 'Previsão'])
            ax.set_title('Preço de Fechamento - {}'.format(ativo))
            ax.set(xlabel='Tempo (ano)', ylabel='Preço ($)')
            nameGraphic = '{}.jpg'.format(ativo)
            fullname = os.path.join(outdir, nameGraphic)
            plt.savefig(fullname)
            plt.close(fig)
        #------------------------------------------------------------------------------------------------------------

        
        #--------Popula tabela de Log Retorno------------------------------------------------------------------------
        for n in range(len(forecast)):
            tableLogRet.loc[lengthTable-n-1, ativo] = \
            np.log(TimeframeSPAux.iloc[lengthTable-steps-n-1, -1] / TimeframeSPAux.iloc[lengthTable-steps-n-1-logRetPeriod, -1])
        #------------------------------------------------------------------------------------------------------------

# Resultados

In [ ]:
# Cria tabela de Previsão

update = pd.DataFrame(index = pd.to_datetime(forecast), columns = ativos) \
    .reset_index().rename(columns={'index': 'Dia'})

tablePrevision = pd.concat([SP500_close, update], axis = 0, ignore_index = True).set_index('Dia')

tablePrevision.index = pd.to_datetime(tablePrevision.index)

In [ ]:
# Cria tabela de Log-Retorno vazia 

index_data = pd.to_datetime(SP500_close['Dia'].append(pd.Series(forecast)))

tableLogRet = pd.DataFrame(index = index_data,
                           columns = ativos).reset_index().rename(columns={'index': 'Dia'})

tableLogRet['Dia'] = tableLogRet['Dia'].apply(lambda date: date.strftime('%d/%m'))

In [ ]:
for nameSetor, setor in setores.items():

    if trainModel:
        X_treino, X_teste, y_treino, y_teste = preprocessingdata(steps, SP500_close, setor)
        model = create_model()

        model.fit(x = X_treino,
                  y = y_treino,
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = verbose,
                  validation_data = (X_teste, y_teste),
                  callbacks = callbacks)
        #---------Save Model---------------------------
        model.save('saveModel/{}/'.format(nameSetor),
                   overwrite=True,
                   include_optimizer=True,
                   save_format = 'tf')
        #----------------------------------------------

        scoreTrain = model.evaluate(X_treino, y_treino)
        scoreTest = model.evaluate(X_teste, y_teste)

        print('\n\nErro quadrático médio em dados de treinamento: {:.5f}\n\nErro quadrático médio em dados de teste: {:.5f}\n\n'\
            .format(scoreTrain, scoreTest))

        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)
        
    else:
        model = load_model('saveModel/{}/'.format(nameSetor))
        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)


### Salva Métricas

In [ ]:
# Salva tabela de previsão

outdirP = './previsao/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirP):
    os.mkdir(outdirP)

fullnameP = os.path.join(outdirP, 'previsao.csv')

tablePrevision.to_csv(fullnameP, index = True, decimal = '.', sep=',')  

In [ ]:
# Salva tabela Entregável do Log-Retorno no padrão

outdirLR = './logRetorno/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirLR):
    os.mkdir(outdirLR)

fullnameLR = os.path.join(outdirLR, 'logRetorno.csv')

tableLogRet.iloc[-len(forecast):, :].to_csv(fullnameLR, index = False, decimal = '.', sep=',')       